In [1]:
import tensorflow as tf
import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import dqn, ppo

from env import *
from model import *
from action_distribution import *

2023-03-14 19:29:37.703893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 19:29:37.823719: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-14 19:29:38.305564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/abdul/miniconda3/lib/
2023-03-14 19:29:38.305606: W tensorflow/compiler/xla/stre

In [2]:
%load_ext autoreload
%autoreload 2

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-03-14 19:29:39.453865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 19:29:39.457410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 19:29:39.457520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 19:29:39.827608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 19:29:39.827858: I tensorflow/compiler/xla/stream_executo

True

r/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 19:29:39.828117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 1476 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-03-14 19:29:41,639	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.3.0


In [4]:
n_disks = 3
env_config = {
    'n_disks': n_disks,
    'max_n_steps': 20000 #2 ** (n_disks + 2)
}

ENV_NAME = 'hanoi_tower'
register_env(ENV_NAME, create_env)

config = ppo.PPOConfig()
config = config.training(lr=1e-3,
                         gamma=0.9,
                         train_batch_size=512 * 2,
                         sgd_minibatch_size=256 * 1,
                         num_sgd_iter=16 * 1)
config = config.rollouts(num_rollout_workers=8,
                         num_envs_per_worker=8)
config = config.environment(HanoiTower, 
                            env_config=env_config)
config = config.resources(num_gpus=1)
config = config.framework('tf2')

model_config = {
    'n_hiddens': 128
}

config.model.update({
    'custom_model': MODEL_NAME,
    'custom_model_config': model_config,
    'custom_action_dist': ACTION_DIST_NAME,
    'vf_share_layers': False
})

config.seed = 19
agent = config.build()

2023-03-14 19:29:42,598	INFO algorithm_config.py:2888 -- Executing eagerly (framework='tf2'), with eager_tracing=tf2. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2023-03-14 19:29:42,612	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


In [ ]:
config.to_dict()

In [ ]:
for _ in range(50):
    res = agent.train()
agent.save()

In [ ]:
env = create_env(env_config)
obs, _ = env.reset()
obss =  []
episode_reward = 0
done = False
while not done:
    action = agent.compute_single_action(obs, explore=False)
    print(obs, action)
    obss.append(obs)
    obs, reward, done, terminated, info = env.step(action)
    episode_reward += reward


In [ ]:
obs, episode_reward

In [ ]:
env = create_env(env_config)
obs, _ = env.reset()
mask = tf.reduce_any(obs > 0, axis=-1)
print('heyyyyy', mask)

In [ ]:
obs = env.step([0,1])
mask = tf.reduce_any(obs[0] > 0, axis=-1)
print('heyyyyy', mask)

In [ ]:
obs = env.step([0,1])


In [ ]:
policy = agent.get_policy()
model = policy.model
model

In [ ]:
model.flatten[0].base_model.summary()

In [ ]:
model.logits_and_value_model.summary()

In [ ]:
new = np.array(obss)
new.shape

In [ ]:
mask = tf.reduce_any(new> 0, axis=-1)
mask[:20]

In [ ]:
model.model.from_model.summary()

In [ ]:
model.model.to_model.summary()

In [ ]:
env = HanoiTower(config=env_config)

In [ ]:
env.action_space.sample()

In [ ]:
obs = env.reset()
env.observation_space.contains(obs)

In [ ]:
obs

In [ ]:
env.step([0, 1])